In [46]:
from sympy.ntheory import isprime

Ring_start is the first value of a new ring. Ring_end is the last value of a new ring. 
For the inner point we have:
   $$ r_s (\cdot) = r_e (\cdot) = 1  $$

For the $1st$ layer we have:
   $$ r_s (0) = 2 \\ r_e (0) = r_s (0) + 6(k+1) - 1 \\ r_e (1) = 7$$

For $k>1$, we will define it to be the k-th ring:
   $$ 
   r_s (k) = r_s (k-1) + 6k  \\$$
   $$
   \therefore r_s (k) = 3k(k + 1) + 2 \\
   $$
   $$ and $$
   $$
   r_e (k) = r_s (k+1) - 1 \\
   r_e (k) = 3(k+1)(k+2) + 1 \\$$
   $$
   \therefore r_e (k) = 3k^2 + 9k + 7 \\
   $$
   

As tile $r_s (k)\mod 2 = 0$, and $r_s (k)$ will always be connected to $r_s (k-1)$ and $r_s (k+1)$, neither of these differences can be a prime number. Likewise, the tile $r_s (k)$ will always be connected to the tile $r_s (k) + 1$ and this difference will always be equal to $1$. 

Thus we must check the other three connections to see if the differences are prime. 
   $$(1) \quad a = r_s (k+1) + 1 - r_s (k) \\
   = 3k^2 + 9k + 9 - 3k^2 + 3k - 2 \\
   \therefore a = 6k + 7 \\
   $$
   $$
   (2) \quad b = r_e (k) - r_s (k) \\
   = r_s (k+1)- r_s (k) - 1 = a - 2 \\
   \therefore b = 6k + 5
   $$
   $$
   (3) \quad c = r_e (k+1) - r_s (k)\\
   = r_s (k+2) - r_s (k) - 1 \\
   \therefore c = 12k + 17
   $$

We have a similar scenario for tile $r_e (k)$, ending up with:
$$(1) \quad x = 6k + 11$$
$$(2) \quad y = 6k + 5$$
$$(3) \quad z = 12k + 5$$
   

In [61]:
# ring_start is the first value of a new ring. Ring_end is the last value of a new ring. 
# For the k-th ring: 
#   ring_start = (6*k + 2)
#   ring_end = (6*k - 1) + ring_start
# We also find that: 
#   ring_start(k) - ring_start(k-1) = 

def find_nth_diff(limit):
    count = 0
    k = 1
    while True:
        z = 6*k + 5
        if isprime(z):
            x_1 = 6*k + 7
            x_2 = 12*k + 17
            if isprime(x_1) and isprime(x_2):
                count += 1
                if count >= limit:
                    return (3*k*(k+1) + 2)

            
            y_1 = 6*k + 11
            y_2 = 12*k + 5
            if isprime(y_1) and isprime(y_2):
                count += 1
                if count >= limit:
                    return (3*(k**2) + 9*k + 7)
                
        k +=1

    

In [64]:
find_nth_diff(2000)

14551069141

In [243]:
def check_node_tile(ele: np.ndarray, i: int, j: int):
    e = int(ele[i,j+1])
    if isprime(e + 1) and isprime(e - 1):
        return True
    return False

def check_layer_start_tile(ele: np.ndarray, i: int, j: int):
    check_1 = isprime(int(ele[i,j+1] + ele[i,j+2])-1)
    check_2 = check_node_tile(ele, i, j)
    if check_1 and check_2:
        return True
    return False

def check_layer_end_tile(diff: np.ndarray, node:np.ndarray, j: int):
    check_1 = isprime(int(diff[j+1] -1))
    check_2 = isprime(int(diff[j] -1))
    check_3 = isprime(int(node[j] - node[j-2] + 1))
    if check_1 and check_2 and check_3:
        return True
    return False

def return_non_zero_list(arr1: np.ndarray, arr2: np.ndarray):
    return sorted(arr1[np.nonzero(arr1)].tolist() + arr2[np.nonzero(arr2)].tolist())


In [279]:
def find_nth_tile(n, stepSize=40):
    layer_len = 0
    while True:
        layer_len += stepSize
        node_len = 6
        diff_array = np.zeros((node_len, layer_len))
        node_array = np.zeros((node_len, layer_len))
        ind_array = np.zeros((node_len, layer_len))



        # Tile array
        layer_inc = 0
        diff_array[0,0] = 1
        for j in range(1,layer_len):
            layer_inc += 6
            diff_array[0,j] = layer_inc

        node_inc = 0
        for j in range(layer_len):
            node_inc += 1
            for i in range(1,node_len):
                diff_array[i,j] = diff_array[i-1,j]+node_inc


        # Node array
        layer_inc = 0
        node_array[0,0] = 2
        for j in range(1,layer_len):
            layer_inc += 6
            node_array[0,j] += node_array[0,j-1] + layer_inc

        node_inc = 0
        for j in range(layer_len):
            node_inc += 1
            for i in range(1,node_len):
                node_array[i,j] = node_array[i-1,j] + node_inc


        # Indicator array
        ind_array[0,0] = 1
        for j in range(layer_len-2):
            if check_layer_start_tile(diff_array,0,j):
                ind_array[0,j] = 1

        for j in range(layer_len-1):
            for i in range(1,node_len):
                if isprime(int(diff_array[i,j])):
                    if check_node_tile(diff_array,i,j):
                        ind_array[i,j+1] = 1


        end_diff_array = diff_array[0,:]
        end_node_array = node_array[0,:] - 1 
        end_ind_array = np.zeros((layer_len))

        end_ind_array[0] = 1
        for j in range(2,layer_len-1):
            if check_layer_end_tile(end_diff_array, end_node_array, j):
                end_ind_array[j] = 1


        tiled_array = node_array * ind_array
        end_tiled_array = end_node_array * end_ind_array

        res = [int(x) for x in return_non_zero_list(tiled_array, end_tiled_array)]

        if len(res) > n:
            return res[n-1]

In [280]:
n = 2166
stepSize = 25_000

res = find_nth_tile(n, stepSize)
print(f'The {n}th tile in the series is: {res}')

The 2166th tile in the series is: 14516824220


In [262]:
print(res - 2415336751) 

2895067
